In [ ]:
import warnings;warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.0MB 6.6MB/s 
     |████████████████████████████████| 1.1MB 41.5MB/s 
     |████████████████████████████████| 3.0MB 42.7MB/s 
     |████████████████████████████████| 890kB 41.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f66959b37eefff5be520ecc337aeb45f486e0303fcafab50eaf0bf512fdad813
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
from torch.utils.data import Dataset,DataLoader

In [ ]:
from tqdm import tqdm_notebook,tnrange

In [ ]:
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
!pip install bidict

In [ ]:
from bidict import bidict
import math

In [ ]:
import pandas as pd

In [ ]:
import joblib

In [ ]:
input_file_path = "/content/gdrive/My Drive/Colab Notebooks/U BERT/data.csv"

In [ ]:
from pathlib import Path

### Split Data into Train and Test

In [ ]:
data = pd.read_csv(input_file_path)

NameError: ignored

In [ ]:
train = pd.DataFrame()
test = pd.DataFrame()
for grp,df in data.groupby(['reviewerID']):
    df = df.sample(frac=1)
    t = int(len(df)*0.8)
    train = train.append(df.iloc[:t])
    test = test.append(df.iloc[t:])
train  = train.dropna().reset_index(drop=True)
test  = test.dropna().reset_index(drop=True)

In [ ]:
len(train),len(test)

(7701, 2526)

In [ ]:
train_fpath = "/content/gdrive/My Drive/Colab Notebooks/U BERT/train.csv"

In [ ]:


train.to_csv(train_fpath,index=False)

In [ ]:
test_fpath = "/content/gdrive/My Drive/Colab Notebooks/U BERT/test.csv"

In [ ]:


test.to_csv(test_fpath,index=False)

### Create user to index mappings

In [ ]:
user2idx={}
for user in data.reviewerID.unique():
    user2idx[user]=len(user2idx)
len(user2idx)

NameError: ignored

In [ ]:
user2idx = bidict(user2idx)

In [ ]:
user2idx_fpath = "/content/gdrive/My Drive/Colab Notebooks/U BERT/user2idx.pkl"

In [ ]:
joblib.dump(user2idx,user2idx_fpath)

['/content/gdrive/My Drive/Colab Notebooks/U BERT/user2idx.pkl']

In [ ]:
user2idx= joblib.load(user2idx_fpath)

### Default configurations of BERT

In [ ]:
config = BertConfig()

In [ ]:
config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

### Load Pretrained bert model

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
tokenizer.sep_token, tokenizer.sep_token_id

('[SEP]', 102)

In [ ]:
tokenizer.cls_token, tokenizer.cls_token_id

('[CLS]', 101)

In [ ]:
tokenizer.pad_token, tokenizer.pad_token_id

('[PAD]', 0)

In [ ]:
tokenizer.unk_token, tokenizer.unk_token_id

('[UNK]', 100)

In [ ]:
train.head(n=2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A00625243BI8W1SSZNLMD,B000MVYOZY,jespi59jr,"[0, 0]","I'm now have my music together, great for smal...",5.0,Good for the price!,1362787200,"03 9, 2013"
1,A00625243BI8W1SSZNLMD,B000SAC5PA,jespi59jr,"[0, 0]","I'm a new user, but seem to be a Great product...",5.0,Acoustic Guitar Bag,1353024000,"11 16, 2012"


In [ ]:
train = train[['reviewerID','reviewText','overall']]

In [ ]:
test = test[['reviewerID','reviewText','overall']]

In [ ]:
train.iloc[0]

reviewerID                                A00625243BI8W1SSZNLMD
reviewText    I'm now have my music together, great for smal...
overall                                                       5
Name: 0, dtype: object

In [ ]:
train.iloc[0].reviewerID,train.iloc[0].reviewText,train.iloc[0].overall

('A00625243BI8W1SSZNLMD',
 "I'm now have my music together, great for small space &#34;does it works&#34; limit to one mic, but enogh for me.",
 5.0)

### Dataset class for pretraining
- Returns input_ids, attention mask, user_ids and user rating for training

In [ ]:
class PreTrainingDataset(Dataset):
    def __init__(self,reviews,tokenizer,user2idx,max_len):
        self.reviews = reviews
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.user2idx = user2idx
    
    def __len__(self): return len(self.reviews)
    
    def __getitem__(self,item):
        
        user_id = self.reviews[item][0]
        review_text = self.reviews[item][1]
        score = self.reviews[item][2]
        
        encoding = self.tokenizer.encode_plus(review_text,
                                              add_special_tokens=True,
                                              max_length=self.max_len,
                                              return_token_type_ids=False,
                                              pad_to_max_length=True,
                                              return_attention_mask=True,
                                              return_tensors='pt',
                                              truncation=True)
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'user' : torch.LongTensor([self.user2idx[user_id]]),
            'targets': torch.Tensor([score])
        }
        
        

### Maximum sequence length

In [ ]:
max_len = 128

In [ ]:
ds = PreTrainingDataset(train.to_numpy(),
                  tokenizer,
                  user2idx,
                  max_len
                  )

In [ ]:
val_ds = PreTrainingDataset(test.to_numpy(),
                           tokenizer,
                           user2idx,
                           max_len)

### Dataloader creation

In [ ]:
dl = DataLoader(ds,batch_size=4,shuffle=True)

In [ ]:
val_dl = DataLoader(val_ds,batch_size=4)

### Load pretrained model for reviews encoder

In [ ]:
model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
for data in dl:
    break

In [ ]:
data

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          

In [ ]:
input_ids = data['input_ids']
attention_mask = data['attention_mask']
user_ids = data['user']
targets = data['targets']

In [ ]:
seq_output,pooled_output = model(input_ids=input_ids,attention_mask=attention_mask)

In [ ]:
seq_output.shape, pooled_output.shape

(torch.Size([4, 128, 768]), torch.Size([4, 768]))

### DotProduct Attention
- mentioned as Attn in U-BERT paper

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self,dk):
        super().__init__()
        self.scale = dk**0.5
        
    def forward(self,Q,K,V,att_mask):
        scores = torch.matmul(Q,K.transpose(-2,-1))/self.scale
        
        if att_mask is not None:
            scores.masked_fill_(att_mask,-1e9)
            
        att = torch.softmax(scores,dim=-1)
        context = torch.matmul(att,V)
        return context,att

### GELU activation function

In [ ]:
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

### Feed Forward network
- mention as FFN in paper 

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self,d_model,d_ff):
        super().__init__()
        self.ff1 = nn.Linear(d_model,d_ff)
        self.act = gelu
        self.ff2 = nn.Linear(d_ff,d_model)
        self.layer_norm = nn.LayerNorm(d_model)
        
    def forward(self,x):
        residual = x
        
        x = self.ff1(x)
        x = self.act(x)
        x = self.ff2(x)
        x = self.layer_norm(x + residual)
        return x

In [ ]:
# UI - User (or) Item

### User / Item emebdding
- class named as *UIEmbedding*, because this class is common for both user and item encoder.
- mentioned as *u* in the paper

In [ ]:
class UIEmbedding(nn.Module):
    def __init__(self,user_count,d_model):
        super().__init__()
        self.user_emb = nn.Embedding(user_count,d_model)
        self.ln = nn.LayerNorm(d_model)
        
    def forward(self,x):
        emb = self.user_emb(x)
        return self.ln(emb)

### User / Item Encoder
- Input: User/Item embedding, encoder output from User reviews /Item reviews
- Output: User / Item representation with respect to their reviews
- Refer section 4.2.3 in the paper

In [ ]:
class UIEncoder(nn.Module):
    def __init__(self,user_count,d_model,d_ff):
        super().__init__()
        self.attn = ScaledDotProductAttention(d_model)
        self.user_embedding = UIEmbedding(user_count,d_model)
        self.Wu = nn.Linear(d_model,d_model,bias=False)
        self.laye_norm = nn.LayerNorm(d_model)
        self.ffn = PositionwiseFeedForward(d_model,d_ff)
        
    def forward(self,x,enc_output,user_review_mask):
        emb = self.user_embedding(x)
        emb = self.Wu(emb)
        s_u,att = self.attn(emb,enc_output,enc_output,user_review_mask)
        h_u = self.laye_norm(emb + s_u)
        user_rep = self.ffn(h_u)
        return user_rep
        

In [ ]:
user_count = len(user2idx)
user_count

1429

In [ ]:
config.hidden_size,config.intermediate_size

(768, 3072)

In [ ]:
user_encoder = UIEncoder(user_count,config.hidden_size,config.intermediate_size)

In [ ]:
def to_cuda(x):
    if torch.cuda.is_available():
        return x.cuda()
    return x

### Combine User encoder and reviews encoder

In [ ]:
class PreTrainingModel(nn.Module):
    def __init__(self,bert_model,user_encoder):
        super().__init__()
        self.bert_model = bert_model
        self.user_encoder = user_encoder 
        
    def forward(self,input_ids,attention_mask,user_ids):
        seq_output, pooled_output = self.bert_model(input_ids=input_ids,attention_mask=attention_mask)
        user_rep = self.user_encoder(user_ids,seq_output,(attention_mask==0).unsqueeze(1))
        
        return user_rep

In [ ]:
 pretrain_model = PreTrainingModel(model,user_encoder)

### Final Layer 
- tranform user representation logits to 1-dimensional(rating score)

In [ ]:
class PreTrainStage(nn.Module):
    def __init__(self,pretrain_model,d_model):
        super().__init__()
        self.pretrain_model = pretrain_model
        self.out = nn.Linear(d_model,1)
        
    def forward(self,input_ids,attention_mask,user_ids):
        user_rep = self.pretrain_model(input_ids,attention_mask,user_ids)
        user_score = self.out(user_rep)
        return user_score
    

In [ ]:
p_model =to_cuda(PreTrainStage(pretrain_model,config.hidden_size))

In [ ]:
optimizer = optim.Adam(p_model.parameters(),lr=2e-5)

In [ ]:
loss_fn = nn.MSELoss()

In [ ]:
epochs=10

### Training Loop

In [ ]:
%%time
for epoch in tnrange(epochs):
    dl = DataLoader(ds,batch_size=4,shuffle=True)
    t = tqdm_notebook(dl,total=len(dl),leave=False)
    trn_loss=[]
    val_loss=[]
    for data in t:
        input_ids = to_cuda(data['input_ids'])
        attention_mask = to_cuda(data['attention_mask'])
        user_ids = to_cuda(data['user'])
        targets = to_cuda(data['targets'])
        
        optimizer.zero_grad()
        user_scores = p_model(input_ids,attention_mask,user_ids)
        loss = loss_fn(user_scores.squeeze(),targets.squeeze())
        loss.backward()
        trn_loss.append(loss.item())
        t.set_postfix(loss=loss.item())
        optimizer.step()
        
    trn_loss = sum(trn_loss)/len(trn_loss)
    
    t = tqdm_notebook(val_dl,total=len(val_dl),leave=False)
    with torch.no_grad():
        for data in t:
            input_ids = to_cuda(data['input_ids'])
            attention_mask = to_cuda(data['attention_mask'])
            user_ids = to_cuda(data['user'])
            targets = to_cuda(data['targets'])
            user_scores = p_model(input_ids,attention_mask,user_ids)
            loss = loss_fn(user_scores.squeeze(),targets.squeeze())
            val_loss.append(loss.item())
            t.set_postfix(loss=loss.item())
    val_loss = sum(val_loss)/len(val_loss)
            
    print("Epoch {} Train loss: {} Val loss: {}".format(epoch+1,round(trn_loss,5),round(val_loss,5)))

Epoch 1 Train loss: 0.67138 Val loss: 0.53679


Epoch 2 Train loss: 0.30875 Val loss: 0.45486


Epoch 3 Train loss: 0.17283 Val loss: 0.4598


Epoch 4 Train loss: 0.13089 Val loss: 0.45959


Epoch 5 Train loss: 0.09751 Val loss: 0.43677


Epoch 6 Train loss: 0.07455 Val loss: 0.45281


Epoch 7 Train loss: 0.06662 Val loss: 0.43708


Epoch 8 Train loss: 0.05063 Val loss: 0.47518


Epoch 9 Train loss: 0.04344 Val loss: 0.42348


Epoch 10 Train loss: 0.03128 Val loss: 0.41947

CPU times: user 46min 25s, sys: 14min 31s, total: 1h 56s
Wall time: 1h 1min 20s


In [ ]:
user_model_fpath = "/content/gdrive/My Drive/Colab Notebooks/U BERT/user_encoder.pt"
encoder_model_fpath = "/content/gdrive/My Drive/Colab Notebooks/U BERT/review_encoder.pt"

In [ ]:
torch.save(user_encoder.state_dict(),user_model_fpath)

torch.save(model.state_dict(),encoder_model_fpath)

### Rating Prediction Model

In [ ]:
TRAIN = False # during testing make it False

In [ ]:
train = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/U BERT/train.csv")
max_len = 128
config = BertConfig()
user2idx_fpath = "/content/gdrive/My Drive/Colab Notebooks/U BERT/user2idx.pkl"
user2idx = joblib.load(user2idx_fpath)
user_count = len(user2idx)
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
user_model_fpath = "/content/gdrive/My Drive/Colab Notebooks/U BERT/user_encoder.pt"
encoder_model_fpath = "/content/gdrive/My Drive/Colab Notebooks/U BERT/review_encoder.pt"


In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

### Create Item to index

In [ ]:
item2idx = {}
for i in train.asin.unique():
    item2idx[i]=len(item2idx)
len(item2idx)

899

In [ ]:
item2idx = bidict(item2idx)

In [ ]:
from torch.utils.data import Dataset,DataLoader

In [ ]:
item2idx_fpath = "/content/gdrive/My Drive/Colab Notebooks/U BERT/item2idx.pkl"


In [ ]:
if TRAIN:
    joblib.dump(item2idx,item2idx_fpath)

['/content/gdrive/My Drive/Colab Notebooks/U BERT/item2idx.pkl']

In [ ]:
item2idx = joblib.load(item2idx_fpath)
item_count = len(item2idx)

In [ ]:
item_count

899

### Create User,item pairs
- Model needs to be trained for this pairs

In [ ]:
user_item_pair=[]
for user,df in train[['reviewerID','asin','overall']].groupby('reviewerID'):
    for i,row in df.iterrows():
        user_item_pair.append((user,row.asin,row.overall))

In [ ]:
len(user_item_pair)

7694

In [ ]:
user_item_pair[0]

('A00625243BI8W1SSZNLMD', 'B000PO30QM', 5.0)

### Dataset class for rating prediction model
- returns input_ids and attention_mask for each review

In [ ]:
class RPDataset(Dataset):
    def __init__(self,reviews,tokenizer,ui2idx,max_len):
        self.reviews = reviews
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.ui2idx = ui2idx
    
    def __len__(self): return len(self.reviews)
    
    def __getitem__(self,item):
        
        ui_id = self.reviews[item][0]
        review_text = self.reviews[item][1]
        
        encoding = self.tokenizer.encode_plus(review_text,
                                              add_special_tokens=True,
                                              max_length=self.max_len,
                                              return_token_type_ids=False,
                                              pad_to_max_length=True,
                                              return_attention_mask=True,
                                              return_tensors='pt',
                                              truncation=True)
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
        
        

### Combined dataset class for user and item reviews
- returns user_input_ids, user_attention_mask, item_input_ids, item_attention_mask, user_id, item_id, rating_score

In [ ]:
class UserDataset(Dataset):
    def __init__(self,user_item_pairs,tokenizer,max_len,user2idx,item2idx,review_ds,sample_len):
        self.user_item_pairs = user_item_pairs
        self.user2idx = user2idx
        self.item2idx = item2idx
        self.review_ds = review_ds
        self.sample_len = sample_len
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self): return len(self.user_item_pairs)
    
    def __getitem__(self,item):
        user,item,score = self.user_item_pairs[item]
        
        user_rev = self.review_ds[self.review_ds['reviewerID']==user]
        item_rev = self.review_ds[self.review_ds['asin']==item]
        
        if len(user_rev)>self.sample_len:
            user_rev = user_rev.sample(self.sample_len)
            
        if len(item_rev)>self.sample_len:
            item_rev = item_rev.sample(self.sample_len)
            
        user_ds = RPDataset(user_rev[['reviewerID','reviewText']].to_numpy(),self.tokenizer,self.user2idx,self.max_len)
        item_ds = RPDataset(item_rev[['asin','reviewText']].to_numpy(),self.tokenizer,self.item2idx,self.max_len)
        
        user_dl = next(iter(DataLoader(user_ds,batch_size=len(user_ds),shuffle=True)))
        item_dl = next(iter(DataLoader(item_ds,batch_size=len(item_ds),shuffle=True)))
        
        return {
            'user_dl' : user_dl,
            'item_dl' : item_dl,
            'user' : torch.LongTensor([self.user2idx[user]]),
            'item' : torch.LongTensor([self.item2idx[item]]),
            'score' : torch.Tensor([score])
        }
            
        
        
        
        

In [ ]:
train= train[['reviewerID','asin','reviewText']]

In [ ]:
user_ds = UserDataset(user_item_pair,tokenizer,max_len,user2idx,item2idx,train,10)

In [ ]:
dl = DataLoader(user_ds,batch_size=1)

In [ ]:
for data in dl:
    break

In [ ]:
data['user_dl']

{'attention_mask': tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,


In [ ]:
data['item_dl']

{'attention_mask': tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,


In [ ]:
data['user']

tensor([[1152]])

In [ ]:
data['item']

tensor([[0]])

In [ ]:
data['score']

tensor([[5.]])

### Function for attention mask
- this mask is used for co-matching layer
- masking padded token in item reviews / user reviews, when learning similarities between them.
- Refer co-matching layer

In [ ]:
def get_rev_attn_mask(att_mask,q_len):
    mask = att_mask.eq(0).repeat(q_len,1)
    return mask

### Co-Matching Layer
- exact implementation from paper
- Comprehensive user review representation(1 representation for all user reviews) from all user reviews with respect to item reviews is obtained.
- Comprehensive item review representation(1 representation for all item reviews) from all item reviews with respect to user reviews is obtained.

In [ ]:
class CoMatchingLayer(nn.Module):
    def __init__(self,d_model):
        super().__init__()
        self.u2i_attn = ScaledDotProductAttention(d_model)
        self.i2u_attn = ScaledDotProductAttention(d_model)
        
        self.WM = nn.Linear(2*d_model,d_model)
        
    def forward(self,user_review_enc,item_review_enc,user_mask,item_mask):
        Du,_ = self.u2i_attn(user_review_enc,item_review_enc,item_review_enc,user_mask)
        Di,_ = self.i2u_attn(item_review_enc,user_review_enc,user_review_enc,item_mask)
        
        Mu = F.tanh(self.WM(torch.cat([user_review_enc-Du,user_review_enc*Du],dim=-1)))
        Mi = F.tanh(self.WM(torch.cat([item_review_enc-Di,item_review_enc*Di],dim=-1)))
        
        tu,_ = Mu.max(0,keepdim=True)
        ti,_ = Mi.max(0,keepdim=True)
        
        return tu,ti       

### Rating PRediction model
- user representation from user encoder
- item representation from item encoder
- user review representation from co-matching layer
- item review representation from co-matching layer

All of these 4 representation concatenated together and transformed to rating score.

In [ ]:
class RatingPrediction(nn.Module):
    def __init__(self,user_encoder,item_encoder,user_review_encoder,item_review_encoder,d_model):
        super().__init__()
        self.user_encoder= user_encoder
        self.item_encoder = item_encoder
        self.user_review_encoder = user_review_encoder
        self.item_review_encoder = item_review_encoder
        
        self.co_match_layer = CoMatchingLayer(d_model)
        self.d_model = d_model
        
        self.WR = nn.Linear(4*d_model,1)
        
    def forward(self,user_id,item_id,user_reviews,item_reviews,user_attn_mask,item_attn_mask):
        
        user_review_enc ,_= self.user_review_encoder(user_reviews,user_attn_mask)
        user_review_enc = user_review_enc.view(-1,self.d_model)
        
        user_enc_mask = (user_attn_mask.squeeze(0)==1).view(-1)
        user_rep = self.user_encoder(user_id,user_review_enc,user_enc_mask)
        
        item_review_enc,_ = self.item_review_encoder(item_reviews,item_attn_mask)
        item_review_enc = item_review_enc.view(-1,self.d_model)

        item_enc_mask = (item_attn_mask.squeeze(0)==1).view(-1)
        item_rep = self.item_encoder(item_id,item_review_enc,item_enc_mask)
        
        item_mask = get_rev_attn_mask(user_enc_mask,item_review_enc.shape[0])
        user_mask = get_rev_attn_mask(item_enc_mask,user_review_enc.shape[0])
        
        tu,ti = self.co_match_layer(user_review_enc,item_review_enc,user_mask,item_mask)
        
        rep = torch.cat([user_rep.squeeze(0),tu,item_rep.squeeze(0),ti],dim=-1)
        
        r = self.WR(rep)
        
        return r

In [ ]:
user_encoder = UIEncoder(user_count,config.hidden_size,config.intermediate_size)

In [ ]:
user_encoder.load_state_dict(torch.load(user_model_fpath))

<All keys matched successfully>

In [ ]:
user_review_encoder = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
user_review_encoder.load_state_dict(torch.load(encoder_model_fpath))

<All keys matched successfully>

In [ ]:
item_encoder = UIEncoder(item_count,config.hidden_size,config.intermediate_size)

In [ ]:
item_review_encoder = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
item_review_encoder.load_state_dict(torch.load(encoder_model_fpath))

<All keys matched successfully>

In [ ]:
rp = to_cuda(RatingPrediction(user_encoder,item_encoder,user_review_encoder,item_review_encoder,config.hidden_size))

### Training loop

In [ ]:
opt = optim.Adam(rp.parameters(),lr=1e-5)

In [ ]:
epochs = 5

In [ ]:
loss_fn = nn.MSELoss()

In [ ]:
%%time

for epoch in tnrange(epochs):
    dl = DataLoader(user_ds,batch_size=1)
    t = tqdm_notebook(dl,total=len(dl),leave=False)
    trn_loss=[]
    for data in t:

        user_ids = to_cuda(data['user_dl']['input_ids'].squeeze(0))
        user_mask = to_cuda(data['user_dl']['attention_mask'].squeeze(0))
        user = to_cuda(data['user'])
        item_ids = to_cuda(data['item_dl']['input_ids'].squeeze(0))
        item_mask = to_cuda(data['item_dl']['attention_mask'].squeeze(0))
        item = to_cuda(data['item'])
        target = to_cuda(data['score'])
        
        opt.zero_grad()
        logits = rp(user,item,user_ids,item_ids,user_mask,item_mask)
        loss = loss_fn(logits.squeeze(),target)
        loss.backward()
        opt.step()
        trn_loss.append(loss.item())
        t.set_postfix(loss=loss.item())
    trn_loss = sum(trn_loss)/len(trn_loss)
    print("Epoch: {} Train Loss: {}".format(epoch+1,trn_loss))
        

Epoch: 1 Train Loss: 0.6505033532554656


Epoch: 2 Train Loss: 0.5524903885037119


Epoch: 3 Train Loss: 0.4912892500447186


Epoch: 4 Train Loss: 0.4116023276035877


Epoch: 5 Train Loss: 0.31980623758200405

CPU times: user 3h 53min 19s, sys: 1h 39min 25s, total: 5h 32min 44s
Wall time: 5h 37min 16s


### Save & load model

In [ ]:
 rp_model_fpath = "/content/gdrive/My Drive/Colab Notebooks/U BERT/rp_model.pt"

In [ ]:
if TRAIN:
    torch.save(rp.state_dict(),rp_model_fpath)

In [ ]:
rp.load_state_dict(torch.load(rp_model_fpath))

<All keys matched successfully>

### Test prediction

In [ ]:
test = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/U BERT/test.csv")

### Create tokens_map
- testing is bit different from training dataset creation
- tokens map is dictionary which contains input_ids and attention_mask for all revoews

In [ ]:
tokens_map={}
for i,row in test.iterrows():
    encoding = tokenizer.encode_plus(row.reviewText,
                                      add_special_tokens=True,
                                      max_length=max_len,
                                      return_token_type_ids=False,
                                      pad_to_max_length=True,
                                      return_attention_mask=True,
                                      return_tensors='pt',
                                      truncation=True)
    tokens_map[i] = encoding

In [ ]:
len(tokens_map)

2533

### Create test map
- for (user, item) pair do
    - take all user reviews except his/her review for this item
    - take all item reviews(i.e, reviews from other users for this item), except user review for this item 


In [ ]:
test_map={}
for i,row in test.iterrows():
    users = test[test.reviewerID==row.reviewerID]
    items = test[test.asin==row.asin]
    if len(users)>=1 and len(items)>=1:
        user = [index for index in users.index if index!=i]
        item = [index for index in items.index if index!=i]
        if len(user)>=1 and len(item)>=1:
            test_map[(user2idx[row.reviewerID],item2idx[row.asin])]={
                'user':user,
                'item':item
            }

KeyError: ignored

### TestDataset class
- returns user_input_ids, item_input_ids, user_id, item_id, user_attention_mask,item_attention_mask

In [ ]:
class TestDataset(Dataset):
    def __init__(self,test_map,tokens_map,sample_len):
        self.test_map = test_map
        self.tokens_map = tokens_map
        self.user_item_pair = list(test_map.keys())
        self.sample_len = sample_len
        
    def __len__(self): return len(self.test_map)
    
    def __getitem__(self,i):
        user,item = self.user_item_pair[i]
        user_rev = self.test_map[(user,item)]['user']
        if len(user_rev)>self.sample_len:
            user_rev = user_rev[:self.sample_len]
        item_rev = self.test_map[(user,item)] ['item']
        if len(item_rev)>self.sample_len:
            item_rev = item_rev[:self.sample_len]
        user_ids=[]
        user_att_mask=[]
        for idx in user_rev:
            user_ids.append(self.tokens_map[idx]['input_ids'].squeeze(0))
            user_att_mask.append(self.tokens_map[idx]['attention_mask'].squeeze(0))
        item_ids=[]
        item_att_mask=[]
        for idx in item_rev:
            item_ids.append(self.tokens_map[idx]['input_ids'].squeeze(0))
            item_att_mask.append(self.tokens_map[idx]['attention_mask'].squeeze(0))
            
        return {
            'user_ids':torch.stack(user_ids),
            'user_att_mask':torch.stack(user_att_mask),
            'item_ids':torch.stack(item_ids),
            'item_att_mask':torch.stack(item_att_mask),
            'user':torch.LongTensor([user]),
            'item':torch.LongTensor([item])
        }

In [ ]:
test_ds = TestDataset(test_map,tokens_map,10)

In [ ]:
test_dl = DataLoader(test_ds,batch_size=1)

### Prediction loop

In [ ]:
%%time

t = tqdm_notebook(test_dl,total=len(test_dl),leave=True)
output=[]
with torch.no_grad():
    for data in t:
        user_ids = to_cuda(data['user_ids'].squeeze(0))
        user_mask = to_cuda(data['user_att_mask'].squeeze(0))
        user = to_cuda(data['user'])
        item_ids = to_cuda(data['item_ids'].squeeze(0))
        item_mask = to_cuda(data['item_att_mask'].squeeze(0))
        item = to_cuda(data['item'])

        logits = rp(user,item,user_ids,item_ids,user_mask,item_mask)
        output.append((user.item(),item.item(),logits.item()))



CPU times: user 54.5 s, sys: 9.4 s, total: 1min 3s
Wall time: 1min 4s


In [ ]:
len(output)

1782

In [ ]:
output[1]

(1242, 8, 5.1319074630737305)

### Slice needed columns from test dataframe

In [ ]:
test = test[['reviewerID','asin','reviewText','overall']]

### convert output for dataframe

In [ ]:
predictions = pd.DataFrame(output,columns=['reviewerID','asin','PredictionScore'])
predictions.head()

,reviewerID,asin,PredictionScore
0,1152,1,5.171024
1,1242,8,5.131907
2,1242,87,5.046658
3,318,10,5.155452
4,318,11,5.156164


### Convert user id, item id to actual user name, item name

In [ ]:
user2idx = bidict(user2idx)

In [ ]:
predictions.reviewerID = predictions.reviewerID.apply(lambda x: user2idx.inv[x])
predictions.asin = predictions.asin.apply(lambda x: item2idx.inv[x])

In [ ]:
predictions.head(n=2)


,reviewerID,asin,PredictionScore
0,A00625243BI8W1SSZNLMD,B000SAC5PA,5.171024
1,A10044ECXDUVKS,B002GYWBIM,5.131907


### Merge 2 dataframe prediction and test
- to compare prediction score and actual score

In [ ]:
predictions.PredictionScore = predictions.PredictionScore.apply(lambda x: x if x<5 else 5)

In [ ]:
predictions = pd.merge(test,predictions,on=['reviewerID','asin'])
predictions.head(n=2)

,reviewerID,asin,reviewText,overall,PredictionScore
0,A00625243BI8W1SSZNLMD,B000SAC5PA,"I'm a new user, but seem to be a Great product...",5.0,5.0
1,A10044ECXDUVKS,B002GYWBIM,Never has 15 watts sounded so loud and clear. ...,5.0,5.0


In [ ]:
output_fpath = "/content/gdrive/My Drive/Colab Notebooks/U BERT/predictions.csv"

In [ ]:
predictions.to_csv(output_fpath,index=False)

### Metrics

In [ ]:
Y = predictions['overall']
Y_preds = predictions['PredictionScore']

##### RMSE


$RMSE = \large{\sqrt{\frac{\sum_{i=1}^{N}(Y-Ypreds)^2}{N}}}$

In [ ]:
rmse = np.sqrt(np.power(Y-Y_preds,2).sum()/len(Y))

In [ ]:
rmse

0.9885336258793279

## Custom input TEST

In [ ]:
test = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/U BERT/test.csv")

In [ ]:
user_id = "A00625243BI8W1SSZNLMD"
item_id = "B000SAC5PA"

In [ ]:
## List of user reviews

In [ ]:
user_reviews = [
    'My guitar, just Love it! This stand is exactly what I hoped it would be. It is all I want or need from a stand.',
     "I'm a new user, but seem to be a Great product!!!! Can't complaint!!! I finally got it and saw how great it is. It has plenty of roomy pockets, great padding, and is put together with great quality."
]

In [ ]:
## List of item reviews

In [ ]:
item_reviews = [
    'I was dissapointed to find out that you can only use one of theeffects at a time, good though they were. Too expensive foronly one effect at a time. Returned it.',
 'Works as advertised but the keys are stiff. Pads take too much presure to activate. Nice that I dont need another interface to connect it to my laptop.After the first few uses the USB socket broke and fell off into the unit rendering it useless. Wrote to dealer that was listed as selling it to me thru Amazon only to have them tell me the wrong seller was listed. I had registered it with AKAI already so I complained to them and they sent me a new one. I had to pay shipping back the old one to them even though it was a defective product.Not recommending this product.',
 'This was a great purchase.  My son found exactly what he needed and at a price he could afford.  He would definitely purchase from here again.'
]

In [ ]:
user_rev=[]
user_attn=[]
for review_text in user_reviews:
    d = tokenizer.encode_plus(review_text,
                          add_special_tokens=True,
                          max_length=max_len,
                          return_token_type_ids=False,
                          pad_to_max_length=True,
                          return_attention_mask=True,
                          return_tensors='pt',
                          truncation=True)
    user_rev.append(d['input_ids'].squeeze())
    user_attn.append(d['attention_mask'].squeeze())

In [ ]:
user_rev = to_cuda(torch.stack(user_rev,dim=0))
user_attn = to_cuda(torch.stack(user_attn,dim=0))

In [ ]:
user_rev.shape,user_attn.shape

(torch.Size([2, 128]), torch.Size([2, 128]))

In [ ]:
item_rev=[]
item_attn=[]
for review_text in item_reviews:
    d = tokenizer.encode_plus(review_text,
                          add_special_tokens=True,
                          max_length=max_len,
                          return_token_type_ids=False,
                          pad_to_max_length=True,
                          return_attention_mask=True,
                          return_tensors='pt',
                          truncation=True)
    item_rev.append(d['input_ids'].squeeze())
    item_attn.append(d['attention_mask'].squeeze())

In [ ]:
item_rev = to_cuda(torch.stack(item_rev,dim=0))
item_attn = to_cuda(torch.stack(item_attn,dim=0))

In [ ]:
item_rev.shape,item_attn.shape

(torch.Size([3, 128]), torch.Size([3, 128]))

In [ ]:
user = to_cuda(torch.LongTensor([[user2idx[user_id]]]))

In [ ]:
item = to_cuda(torch.LongTensor([[item2idx[item_id]]]))

In [ ]:
with torch.no_grad():
    logits = rp(user,item,user_rev,item_rev,user_attn,item_attn)

In [ ]:
score = logits.squeeze().item()

In [ ]:
score

5.302947998046875